![](./images/07_00.jpg)

* Một công ty quảng cáo trên website đang muốn xây dựng một logistic regression model để dự đoán xem các khách hàng của họ liệu có **rời đi** _(tức là ngừng sử dụng dịch vụ của họ)_ trong tương lai hay ko để họ có thể đề ra những chính sách cần thiết cho khách hàng của họ. Họ có một tập dữ liệu là `customer_churn.csv`, dưới đây là các đặc trưng:
  * `Name`: tên khách hàng.
  * `Age`: tuổi của khách hàng.
  * `Total_Purchase`: tổng số tiền mà khách hàng này đã cúng cho công ty quảng cáo.
  * `Years`: số năm mà khách hàng này gắn bó vs công ty.
  * `Num_sites`: số lượng website mà khách hàng này đặt quảng cáo.
  * `Onboard_date`: ngày mà khách hàng này bắt đầu sử dụng dịch vụ của công ty quảng cáo.
  * `Location`: địa chỉ khách hàng.
  * `Company`: tên công ty.
  * `Account_Manager`: bằng $1$ nếu khách hàng có rủi ro cao sẽ rời bỏ công ty, ngược lại là $0$.
  * `Churn`: rời bỏ công ty $1$ và ngược lại là $0$.
* Khi xây dựng xong model, nhớ đánh giá model này, kiểm tra model bằng dữ liệu mới trong file `new_customer.csv`. Công ty quảng cáo muốn biết rằng khách hàng nó có khả năng sẽ rời bỏ công ty họ.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.my_pyspark import *

In [3]:
spark = MyPySpark(session=True)

# 1. Đọc dữ liệu

In [4]:
customers_data_path: str = r"./data/customer_churn.csv"

In [5]:
customers_data = spark.readFile(customers_data_path)

* Schema 

In [6]:
customers_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



* Thống kê mô tả

In [7]:
customers_data.describe().toPandas()

,summary,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,count,900,900,900,900,900,900,900,900,900,900
1,mean,None,41.81666666666667,10062.82403333334,0.4811111111111111,5.27315555555555,8.587777777777777,None,None,None,0.16666666666666666
2,stddev,None,6.127560416916251,2408.644531858096,0.4999208935073339,1.274449013194616,1.7648355920350969,None,None,None,0.3728852122772358
3,min,Aaron King,22.0,100.0,0,1.0,3.0,2006-01-02 04:16:13,"00103 Jeffrey Crest Apt. 205 Padillaville, IA ...",Abbott-Thompson,0
4,max,Zachary Walsh,65.0,18026.01,1,9.15,14.0,2016-12-28 04:07:38,Unit 9800 Box 2878 DPO AA 75157,"Zuniga, Clark and Shaffer",1


> **Nhận xét**:
> * Tập data có 900 sample.
> * Ko có missing value
> * Feature `Onboard_date` cần chuẩn lại data type.
> * Mặc dù cho chênh lệch thang đo lớn ở feature `Total_Purchase` so với các feature khác, nhưng với logistic regression model thì ta ko cần thiết phải normalize hay standardize dữ liệu.

* Xem qua dữ liệu

In [8]:
customers_data.show(3, False, True)

-RECORD 0-------------------------------------------------------------
 Names           | Cameron Williams                                   
 Age             | 42.0                                               
 Total_Purchase  | 11066.8                                            
 Account_Manager | 0                                                  
 Years           | 7.22                                               
 Num_Sites       | 8.0                                                
 Onboard_date    | 2013-08-30 07:00:40                                
 Location        | 10265 Elizabeth Mission Barkerburgh, AK 89518      
 Company         | Harvey LLC                                         
 Churn           | 1                                                  
-RECORD 1-------------------------------------------------------------
 Names           | Kevin Mueller                                      
 Age             | 41.0                                               
 Total

# 2. Chuẩn hóa dữ liệu

## 2.1. Chuyển feature `Onboard_date` thành `DateType()` datatype

In [9]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, udf
from datetime import datetime

* User define function

In [10]:
standard_Onboard_date = udf(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), DateType())

* Tạo feature mới có tên là `Onboard_date_sd` với giá trị đã dc chuẩn hóa sang `DateType()` từ feature `Onboard_date`

In [11]:
customers_data = customers_data.withColumn('Onboard_date_sd', standard_Onboard_date(col('Onboard_date')))

In [12]:
show_features = customers_data.columns.copy()
show_features.remove('Onboard_date_sd')

In [13]:
show_features

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

# 3. Chuyển đổi dữ liệu
* Lựa chọn thuộc tính:
  * Predictor variables: `Age`, `Total_Purchase`, `Account_Manager`, `Years`, `Num_Sites`, `Onboard_date_sd`
  * Target varible: `Churn`

<hr>

* Indexer feature `Onboard_date_sd` theo tháng

In [14]:
from pyspark.sql.functions import month
from modules.my_spark_logistic_regression import *

In [15]:
customers_data = customers_data.withColumn('Month', month('Onboard_date_sd'))

In [16]:
customers_data.show(3, False, True)

-RECORD 0-------------------------------------------------------------
 Names           | Cameron Williams                                   
 Age             | 42.0                                               
 Total_Purchase  | 11066.8                                            
 Account_Manager | 0                                                  
 Years           | 7.22                                               
 Num_Sites       | 8.0                                                
 Onboard_date    | 2013-08-30 07:00:40                                
 Location        | 10265 Elizabeth Mission Barkerburgh, AK 89518      
 Company         | Harvey LLC                                         
 Churn           | 1                                                  
 Onboard_date_sd | 2013-08-30                                         
 Month           | 8                                                  
-RECORD 1-------------------------------------------------------------
 Names

* Khởi tạo list `predictor_fts` chứa các predictor variables và `target_ft` chứa target variable với feature `Onboard_date_sd` thay bằng `Month`

In [17]:
predictor_fts = ['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Month']
target_ft = 'Churn'

In [18]:
log_model = MySparkLogisticRegression(customers_data, predictor_fts, target_ft)

* Assembler

In [19]:
log_model.prepareData()

In [20]:
log_model.df_data.show(3, False)

+---------------------------------+-----+
|features                         |Churn|
+---------------------------------+-----+
|[42.0,11066.8,0.0,7.22,8.0,8.0]  |1    |
|[41.0,11916.22,0.0,6.5,11.0,8.0] |1    |
|[38.0,12884.75,0.0,6.67,12.0,6.0]|1    |
+---------------------------------+-----+
only showing top 3 rows



# 4. Xây dựng model

In [21]:
log_model.buildModel()

# 5. Đánh giá kết quả

## 5.1. Đánh giá trên train data

In [22]:
log_model.evaluateTrainData()

,Metric,Value,Range,Model performance
0,Accuracy,0.895556,"[0, 1]",high is better
1,Area under ROC,0.908053,"[0.5, 1]",high is better
2,Presicion,0.888688,"[0, 1]",high is better
3,Recall,0.895556,"[0, 1]",high is better
4,F1-Score,0.892109,"[0, 1]",high is better


> **Nhận xét**:
> * Các giá trị đánh giá rât tốt, đa đạt ngưỡng hoặc hơn $90\%$.

## 5.2. Đánh giá trên test data (ko có dữ liệu test data)
# 6. Dự đoán cho tập dữ liệu mới
## 6.1. Đọc dữ liệu

In [23]:
customers_test_data_path = r"./data/new_customers.csv"

In [24]:
customers_test_data = spark.readFile(customers_test_data_path)

## 6.2. Chuẩn hóa và chuyển đổi dữ liệu

In [25]:
customers_test_data = customers_test_data.withColumn('Onboard_date_sd', standard_Onboard_date(col('Onboard_date')))

In [26]:
customers_test_data = customers_test_data.withColumn('Month', month('Onboard_date_sd'))

In [27]:
customers_test_data.show(3, False, True)

-RECORD 0--------------------------------------------------------------------
 Names           | Andrew Mccall                                             
 Age             | 37.0                                                      
 Total_Purchase  | 9935.53                                                   
 Account_Manager | 1                                                         
 Years           | 7.71                                                      
 Num_Sites       | 8.0                                                       
 Onboard_date    | 2011-08-29 18:37:54                                       
 Location        | 38612 Johnny Stravenue Nataliebury, WI 15717-8316         
 Company         | King Ltd                                                  
 Onboard_date_sd | 2011-08-29                                                
 Month           | 8                                                         
-RECORD 1-------------------------------------------------------

## 6.3. Dự đoán cho `customers_test_data`

In [28]:
predicted_customers_test_data = log_model.predict(customers_test_data)

In [29]:
predicted_customers_test_data.show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[37.0,9935.53,1.0...|[2.22343461620499...|[0.90233429706111...|       0.0|
|[23.0,7526.94,1.0...|[-6.2203345510752...|[0.00198463322286...|       1.0|
|[65.0,100.0,1.0,1...|[-3.7611128554502...|[0.02272921092289...|       1.0|
|[32.0,6487.5,0.0,...|[-5.0913101761067...|[0.00611236628931...|       1.0|
|[32.0,13147.71,1....|[1.10051548060227...|[0.75035667867300...|       0.0|
|[22.0,8445.26,1.0...|[-1.6948282326692...|[0.15514193533747...|       1.0|
+--------------------+--------------------+--------------------+----------+

